In [ ]:
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import LancasterStemmer
from nltk.corpus import stopwords
porter = LancasterStemmer()
lemmatizer = WordNetLemmatizer()
from collections import defaultdict
import nltk
from nltk.util import ngrams

from collections import Counter

import gensim
from gensim import models
from gensim.models.phrases import Phrases, Phraser
from gensim import corpora
from pprint import pprint
import spacy
nlp = spacy.load('en_core_web_sm')#try with medium, see if I get better results?
stop_words = stopwords.words("english")

## 1. Load info dataframes and split into three (toc, body, index)

In [ ]:
path = 'cann_info_pages.csv'
df_cann_pages = pd.read_csv(path,index_col=0)
df_cann_pages.head()

In [ ]:
path = 'cann_info_lines.csv'
df_cann_lines = pd.read_csv(path,index_col=0)
df_cann_lines.head()

In [ ]:
df_cann_pages.section_level_1.fillna("No info", inplace=True)
df_cann_lines.section_level_1.fillna("No info", inplace=True)
df_cann_pages.section_level_2.fillna("No info", inplace=True)
df_cann_lines.section_level_2.fillna("No info", inplace=True)
df_cann_pages.section_level_3.fillna("No info", inplace=True)
df_cann_lines.section_level_3.fillna("No info", inplace=True)

In [ ]:
#toc
df_cann_lines_toc=df_cann_lines[(df_cann_lines.section_level_1.str.match("CONTENTS|Contents|contents|Table of Contents|Table of contents"))|(df_cann_lines.section_level_2.str.match("CONTENTS|Contents|contents|Table of Contents|Table of contents"))|(df_cann_lines.section_level_3.str.match("CONTENTS|Contents|contents|Table of Contents|Table of contents"))]

In [ ]:
#body (both versions)
df_cann_pages_body=df_cann_pages[(df_cann_pages.page_number>=19) & (df_cann_pages.page_number<=340)]
df_cann_lines_body=df_cann_lines[(df_cann_lines.page_number>=19) & (df_cann_lines.page_number<=340)]

In [ ]:
#index
df_cann_lines_index=df_cann_lines[(df_cann_lines.section_level_1.str.match(
  "INDEX|index|Index|Subject Index|Name Index|Language Index|Citation Index|General Index|Author Index|Indicies"))|
  (df_cann_lines.section_level_2.str.match("INDEX|index|Index|Subject Index|Name Index|Language Index|Citation Index|General Index|Author Index|Indicies"))|
  (df_cann_lines.section_level_3.str.match("INDEX|index|Index|Subject Index|Name Index|Language Index|Citation Index|General Index|Author Index|Indicies"))]

In [ ]:
#biblio
df_cann_pages_biblio=df_cann_pages[
  (df_cann_pages.section_level_1.str.match("REFERENCES|References|references|Bibliography|bibliography"))|(df_cann_pages.section_level_2.str.match("References|references|Bibliography|bibliography"))|(df_cann_pages.section_level_3.str.match("References|references|Bibliography|bibliography"))]

In [ ]:
df_cann_lines_toc.reset_index(drop=True, inplace=True)
df_cann_pages_body.reset_index(drop=True, inplace=True)
df_cann_lines_body.reset_index(drop=True, inplace=True)
df_cann_lines_index.reset_index(drop=True, inplace=True)
df_cann_pages_biblio.reset_index(drop=True, inplace=True)

In [ ]:
df_cann_lines_index.to_csv("cann_lines_index.csv", encoding = 'utf-8')
df_cann_pages_biblio.to_csv("cann_pages_biblio.csv", encoding = 'utf-8')
#save lines-index df and pages-biblio df to csv

## 2. Cleaning (toc and body -- index and biblio I do it separately later)

In [ ]:
textbook_words=['thus', 'today', 'nowadays', 'actually', 'section', 'indeed', 'every', 'any', 'some', 'example', 'therefore', 'definition', 'introduction', 'conclusion', 'chapter', 'appendix', 'otherwise', 'thing', 'rather', 'instead', 'like', 'since', 'given', 'case', 'hence', 'iff', 'see', 'beyond', 'below', 'above', 'postscript', 'index', 'ensure', 'generally', 'anything', 'something', 'other']


In [ ]:
def clean_text(text_data):
    tokens=word_tokenize(text_data)

    lowercased=[w.lower() for w in tokens]
    
    no_punct=[word for word in lowercased if (word.isalpha() or re.match("[a-z]+-[a-z]+", word))]
    
    no_sw=[w for w in no_punct if w not in stop_words]
    
    clean_tokens=[lemmatizer.lemmatize(word) for word in no_sw]
    
    #no_tw=[w for w in clean_tokens if w not in textbook_words]
    
    #long_words=[w for w in no_tw if len(w)>2]#try it with larger than 1 as well
    
    return (" ").join(clean_tokens)

In [ ]:
df_cann_lines_body["clean_content"] = df_cann_lines_body["content"].apply(clean_text)
df_cann_lines_toc["clean_content"] = df_cann_lines_toc["content"].apply(clean_text)
df_cann_pages_body["clean_content"] = df_cann_pages_body["content"].apply(clean_text)

In [ ]:
df_cann_lines_body.to_csv("cann_lines_body_clean.csv", encoding = 'utf-8')
df_cann_pages_body.to_csv("cann_pages_body_clean.csv", encoding = 'utf-8')
df_cann_lines_toc.to_csv("cann_lines_toc_clean.csv", encoding = 'utf-8')
#save clean dfs to csv